<a href="https://colab.research.google.com/github/TranDung6129/Timetabling_NCKHSV/blob/main/school_timetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libarary and Package

In [463]:
import os
import pandas as pd
import numpy as np
# import pyomo.environ as pyo
# from pyomo.environ import * 
import itertools
# from pyomo.opt import SolverStatus, TerminationCondition
from functools import reduce
from time import time
import random

# Classroom 

In [464]:
import os
import pandas as pd
import numpy as np


class Classroom:

    def __init__(self):
        # Read excel file
        filepath = '/content/TKB SIE ky 20212 (14.4.22)(1).xlsx'
        self.df = pd.read_excel(filepath, sheet_name='Phòng học')
        # Lấy duy nhất hai cột dữ liệu cần thiết 
        self.df = self.df[["Số phòng mới", "Số chỗ ngồi"]]
        
    def get_table(self):
        return self.df
    
    def get_classroom_list(self):
        '''Lấy danh sách các phòng học''' 
        classroom_list = self.df["Số phòng mới"].to_list()
        return classroom_list
    
    def get_classroom_capacity(self, classroom_name):
        '''Lấy sức chứa của từng phòng học theo xác định. 
        classroom_name= ('tên của phòng học muốn lấy sức chứa')'''
        class_capacity_index = self.df.index[self.df['Số phòng mới'] == classroom_name].astype(int)[0]
        return self.df.at[class_capacity_index, "Số chỗ ngồi"]

# Information

In [465]:
import os
import pandas as pd
import numpy as np

class ClassInformation():
    
    def __init__(self):
        filepath = '/content/TKB SIE ky 20212 (14.4.22)(1).xlsx'
        self.df = pd.read_excel(filepath, sheet_name = 'Báo dạy 20212 (2)')
        
        # Thêm cột mã lớp
        new_class_code = []
        self.df['STT theo mã HP'] = self.df['STT theo mã HP'].astype(str).str.zfill(3)
        number_code = self.df["STT theo mã HP"].tolist()
        for i in number_code:
            new_code = "134" + str(i)
            new_class_code.append(new_code)
        self.df.insert(0, "Mã lớp", new_class_code, True)
        
    def get_table(self):
        return self.df
    
    def get_student_number(self, class_code):
        ''' Trả về số sinh viên từng Mã lớp.
        class_code = ('Mã lớp')'''
        class_index = self.df.index[self.df['Mã lớp'] == class_code].astype(int)[0]
        return self.df.at[class_index, 'Số SV lớp cố định']

    def get_class_periods_number(self, class_code):
        '''Trả về số tiết học của một Mã lớp trong kỳ, dựa vào khối lượng'''
        period_index = self.df.index[self.df['Mã lớp'] == class_code].astype(int)[0]
        periods_retrieve = self.df.at[period_index, 'KHỐI LƯỢNG ']
        if int(periods_retrieve[0]) <= 4:
            periods = str(periods_retrieve)[0]
        else: 
            periods = 0
        return periods
    
    def get_participant_class(self, class_code):
        '''Trả về tên của các lớp con tham gia một Mã lớp'''
        participant_class_index  = self.df.index[self.df['Mã lớp'] == class_code].astype(int)[0]
        participant_class = self.df.at[participant_class_index, "Lớp"]
        participant_class_list = list(participant_class.split("+"))
        for i in participant_class_list:
            if i == 'B':
                B_index = participant_class_list.index('B')
                participant_class_list[B_index - 1] = participant_class_list[B_index - 1] + "+" + participant_class_list[B_index]
                participant_class_list.remove("B")
        return participant_class_list
    
    def get_class_code(self):
        class_code = self.df["Mã lớp"].to_list()
        return class_code
    
    def get_class_group(self, class_code):
        '''Trả về tên của các lớp con tham gia một Mã lớp'''
        participant_class_index  = self.df.index[self.df['Mã lớp'] == class_code].astype(int)[0]
        class_group = self.df.at[participant_class_index, "Lớp"]
        return class_group
    
    def get_class_code_each_class_group(self, class_group):
        class_code_index  = self.df.index[self.df['Lớp'] == class_group].astype(int)[0]
        class_code = self.df.at[class_code_index, "Mã lớp"]
        return

    def get_class_group_student_number(self, class_group):
        '''Trả về tên của các lớp con tham gia một Mã lớp'''
        group_student_index  = self.df.index[self.df['Lớp'] == class_group].astype(int)[0]
        class_group_student_number = self.df.at[group_student_index, "Số SV lớp cố định"]
        return class_group_student_number

# DataFrame

In [466]:
# Điều chỉnh DataFrame 
classroom = Classroom()
classroom_df = classroom.df
information = ClassInformation()
information_df = information.df

# G set (G_big and G_small)
An available set which contains all groups of students

In [467]:
start_time = time()
"""Nhập các tập đầu vào của mô hình"""
# Tập hợp group_of_student là tập hợp chứa tất cả các nhóm học sinh
def G_set():
    """Đưa ra tập hợp chứa tất cả các nhóm lớp"""
    group_of_student = []
    for class_code in information.get_class_code():
        group_of_student.append(information.get_class_group(class_code))

    unique_class_group = []
    for class_group in group_of_student:
        if class_group not in unique_class_group:
            unique_class_group.append(class_group)

    group_of_student = unique_class_group
    ## Tạo tập hợp chứa các phần tử rỗng 
    group_of_student_dict = {}
    for n in range(len(group_of_student)):
        group_of_student_dict[f"G{n}"] = []

    ## Nhập dữ liệu các nhóm lớp
    for class_group in range(len(group_of_student)):
        group_of_student_dict[f"G{class_group}"] = group_of_student[class_group]

    ### Chia các lớp thành các phần tử nhỏ hơn
    for key in group_of_student_dict:
        group_of_student_dict[key] = group_of_student_dict[key].split("+")

    ### Chỉnh sửa lại các lớp con 
    for key in group_of_student_dict:
        for i in group_of_student_dict[key]:
            if i == 'B':
                B_index = group_of_student_dict[key].index('B')
                group_of_student_dict[key][B_index - 1] = group_of_student_dict[key][B_index - 1] + "+" + group_of_student_dict[key][B_index]
                group_of_student_dict[key].remove("B")
    return group_of_student_dict
# Tập hợp G_big chứa các nhóm lớp có nhiều hơn hai lớp con từ tập G
def G_big_set(G_set):
    G_big_dict = {}
    for key in G_set:
        if len(G_set[key]) >= 2:
            G_big_dict.update({key: G_set[key]})
    return G_big_dict
# Tập hợp G_small chứa các lớp con (chỉ gồm duy nhất một lớp) từ tập G
def G_small_set(G_set):
    G_small_dict = {}
    for key in G_set:
        if len(G_set[key]) < 2:
            G_small_dict.update({key: G_set[key]})
    return G_small_dict

# Tạo ra tập G
G_set = G_set()
# Lấy tập G lớn từ tập G
G_big_set = G_big_set(G_set)
# Lấy tập G nhỏ từ tập G
G_small_set = G_small_set(G_set)

# C set
An available set which contains all the courses in the semester

In [468]:
# Tập C chứa tất cả các lớp mở trong kỳ
course_dict = {}
for class_code in information.get_class_code():
    course_dict[class_code] = {"Sub": information_df.loc[information.get_class_code().index(class_code), 'TÊN HP']}
## Loại lớp 

# R_set
An available set which contains classrooms grouped by their capacity

In [474]:
# Tập hợp chứa tất cả các phòng học có thể sử dụng
def R_all():
    global classroom
    R_all = []
    for classroom_name in classroom.get_classroom_list():
        R_all.append(classroom_name)
    R_all = set(R_all)
    return R_all
# Tập hợp chứa sức chứa có thể của các phòng
def classroom_capacity(R_all):
    global classroom
    classroom_capacity = []
    for classroom_name in R_all:
        classroom_capacity.append(classroom.get_classroom_capacity(classroom_name))

    classroom_capacity_unique = []

    for i in classroom_capacity:
        if i not in classroom_capacity_unique:
            classroom_capacity_unique.append(i)
    return classroom_capacity_unique

# Tập R là tập hợp chứa các phòng được nhóm lại theo sức chứa
def R_set(classroom_capacity):
    global classroom
    R_set = {}
    for class_capacity in classroom_capacity:
        R_set[class_capacity] = []
    
    for class_capacity in classroom_capacity:
        for classroom_name in classroom.get_classroom_list():
            if classroom.get_classroom_capacity(classroom_name) == class_capacity:
                R_set[class_capacity].append(classroom_name)
    return R_set

# Tạo ra tập R_all
R_all = R_all()
# Tạo ra tập R 
classroom_capacity = classroom_capacity(R_all)
R_set = R_set(classroom_capacity)
# Sắp xếp lại sức chứa của phòng theo thứ tự tăng dần 
classroom_capacity.sort()

In [470]:
print(classroom_capacity)

[0, 14, 32, 36, 45, 50, 60, 84, 120, 132]


# Rg Set
An available set which contains groups of students accompanied by the
classroom’s capacity that fits them


In [471]:
def Rg_set():
    group_of_student = []
    for class_code in information.get_class_code():
        group_of_student.append(information.get_class_group(class_code))

    unique_class_group = []
    for class_group in group_of_student:
        if class_group not in unique_class_group:
            unique_class_group.append(class_group)

    group_student_number = {}
    for class_group in unique_class_group:
        group_student_number[class_group] = information.get_class_group_student_number(class_group)

    for class_group in group_student_number:
        for capacity in classroom_capacity:
            if group_student_number[class_group] < capacity:
                group_student_number[class_group] = capacity
                break
    
    Rg_set = {}
    for i in range(len(G_set)):
        Rg_set[f"G{i}"] = group_student_number[unique_class_group[i]]
    return Rg_set

# Tạo tập Rg 
Rg_set = Rg_set()

# Cg set (Cg_big and Cg_small)

A set which contains groups of students accompanied by the courses that
they need to register in the semester



In [496]:
# Tập C chứa các nhóm lớp, đi cùng với các mã lớp mà nhóm lớp đó sẽ học trong kỳ
def Cg_set():
    Cg_set = {}

    group_of_student = []
    for class_code in information.get_class_code():
        group_of_student.append(information.get_class_group(class_code))

    unique_class_group = []
    for class_group in group_of_student:
        if class_group not in unique_class_group:
            unique_class_group.append(class_group)

    for class_group in unique_class_group:
        temp_list = []
        for i in range(len(information_df)):
            if information_df.iloc[i, 7] == class_group:
                temp_list.append(information_df.iloc[i, 0])
        Cg_set[f"G{unique_class_group.index(class_group)}"] = temp_list
    return Cg_set

# Tập Cg_big chứa các nhóm lớp đi cùng với mã lớp mà nhóm lớp đó sẽ học trong kỳ (số lượng mã lớp lớn hơn hoặc bằng 2)
def Cg_big_set(Cg_set):
    Cg_big_dict = {}
    for key in Cg_set:
        if len(Cg_set[key]) >= 2:
            Cg_big_dict.update({key: G_set[key]})
    return Cg_big_dict

# Tập Cg_small chứa các nhóm lớp đi cùng với mã lớp mà nhóm lớp đó sẽ học trong kỳ (số lượng mã lớp nhỏ hơn 2)
def Cg_small_set(Cg_set):
    Cg_small_dict = {}
    for key in Cg_set:
        if len(Cg_set[key]) < 2:
            Cg_small_dict.update({key: G_set[key]})
    return Cg_small_dict

# Tạo tập Cg 
Cg_set = Cg_set()
# Tạo tập Cg_big 
Cg_big_set = Cg_big_set(Cg_set)
# Tạo tập Cg_small
Cg_small_set = Cg_small_set(Cg_set)

# D set
A set which contains all sessions in week

In [ ]:
D_set = {1, 2, 4, 5, 6, 7, 8, 9, 10}

# P set
A set which contains 6 periods in a session of a day (morning or afternoon), each period lasts 60 min

In [ ]:
P_set = {1, 2, 3, 4, 5, 6}

In [472]:
# from google.colab import drive
# drive.mount('/content/drive')